In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_csv("mobiles.csv")
df.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


### Q1.

In [7]:
df_q1 = df.copy()

In [9]:
stat_mean = df_q1["sales"].mean()
stat_std  = df_q1["sales"].std()
stat_out  = stat_mean + 2 * stat_std
stat_out

146.55150129273215

In [10]:
df_q1_sub = df_q1.loc[df_q1["sales"] > stat_out, ].reset_index(drop = True)
len(df_q1_sub)

16

In [11]:
df_q1_sub["idx"] = (df_q1_sub["ROM"] / 32) + (df_q1_sub["RAM"] / 2) + \
(df_q1_sub["num_front_camera"] + df_q1_sub["num_rear_camera"]) + \
(df_q1_sub["battery_capacity"] / 1000)

In [12]:
df_q1_sub.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,idx
0,Medium,128,6,2,1,4000,4.6,122001,18999,0.09,231.79,14.0
1,Large,128,6,4,2,4500,4.5,267028,15999,0.20,427.22,17.5


In [14]:
round(df_q1_sub["idx"].mean(), 2)

11.01

### Q2.

In [17]:
df_q2 = df.loc[df["num_rear_camera"] != 1, "battery_capacity":]
df_q2 = df_q2.reset_index(drop = True)
df_q2.head(2)

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,2815,4.5,244,57149,0.04,1.39
1,2815,4.6,745,69149,0.02,5.15


In [20]:
df_q2.corr().abs().round(2)

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
battery_capacity,1.00,0.42,0.03,0.50,0.26,0.03
ratings,0.42,1.00,0.19,0.15,0.12,0.23
num_of_ratings,0.03,0.19,1.00,0.26,0.21,0.95
sales_price,0.50,0.15,0.26,1.00,0.10,0.25
discount_percent,0.26,0.12,0.21,0.10,1.00,0.22
sales,0.03,0.23,0.95,0.25,0.22,1.00


In [21]:
df_q2.corr()["sales"].abs().round(2)

battery_capacity    0.03
ratings             0.23
num_of_ratings      0.95
sales_price         0.25
discount_percent    0.22
sales               1.00
Name: sales, dtype: float64

In [22]:
df_q2.corr()

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
battery_capacity,1.000000,-0.424129,0.034902,-0.503019,0.257373,0.025680
ratings,-0.424129,1.000000,0.191655,0.151153,-0.118578,0.226075
num_of_ratings,0.034902,0.191655,1.000000,-0.260279,0.212442,0.949114
sales_price,-0.503019,0.151153,-0.260279,1.000000,-0.098640,-0.247760
discount_percent,0.257373,-0.118578,0.212442,-0.098640,1.000000,0.223471
sales,0.025680,0.226075,0.949114,-0.247760,0.223471,1.000000


In [24]:
df_corr = df_q2.corr().reset_index()
df_corr

,index,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,battery_capacity,1.000000,-0.424129,0.034902,-0.503019,0.257373,0.025680
1,ratings,-0.424129,1.000000,0.191655,0.151153,-0.118578,0.226075
2,num_of_ratings,0.034902,0.191655,1.000000,-0.260279,0.212442,0.949114
3,sales_price,-0.503019,0.151153,-0.260279,1.000000,-0.098640,-0.247760
4,discount_percent,0.257373,-0.118578,0.212442,-0.098640,1.000000,0.223471
5,sales,0.025680,0.226075,0.949114,-0.247760,0.223471,1.000000


In [25]:
df_corr_melt = df_corr.melt(id_vars = "index")
df_corr_melt.head(2)

,index,variable,value
0,battery_capacity,battery_capacity,1.000000
1,ratings,battery_capacity,-0.424129


In [26]:
df_corr_melt.loc[df_corr_melt["index"] != df_corr_melt["variable"], ]

,index,variable,value
1,ratings,battery_capacity,-0.424129
2,num_of_ratings,battery_capacity,0.034902
3,sales_price,battery_capacity,-0.503019
4,discount_percent,battery_capacity,0.257373
5,sales,battery_capacity,0.025680
6,battery_capacity,ratings,-0.424129
8,num_of_ratings,ratings,0.191655
9,sales_price,ratings,0.151153
10,discount_percent,ratings,-0.118578
11,sales,ratings,0.226075


### Q3.

In [54]:
df_q3 = df.copy()
df_q3.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


In [55]:
df_q3_dum = pd.get_dummies(df_q3, columns = ["screen_size"])
df_q3_dum = df_q3_dum.set_index("sales").reset_index()
df_q3_dum.head(2)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,127.52,64,2,1,1,1800,4.5,38645,32999,0.17,0,0,0,0,1
1,1.39,64,4,2,1,2815,4.5,244,57149,0.04,0,0,1,0,0


In [56]:
df_q3_dum.shape

(430, 15)

In [57]:
df_train, df_test = train_test_split(df_q3_dum, train_size = 0.8, 
                                     random_state = 123)
len(df_train), len(df_test)

(344, 86)

In [58]:
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

In [59]:
arr_train_nor[:1]

array([[0.00394753, 0.04761905, 0.18181818, 0.33333333, 0.        ,
        0.42307692, 0.625     , 0.00396262, 0.02607261, 0.09302326,
        0.        , 0.        , 1.        , 0.        , 0.        ]])

In [60]:
df_train_nor = pd.DataFrame(arr_train_nor, columns = df_train.columns)
df_test_nor  = pd.DataFrame(arr_test_nor,  columns = df_test.columns)

In [61]:
ls_k = [3, 5, 7, 9, 11]

k = ls_k[0]
model_knn = KNeighborsRegressor(n_neighbors = k)
model_knn.fit(X = df_train_nor.drop(columns = "sales"),
              y = df_train_nor["sales"])
pred = model_knn.predict(df_test_nor.drop(columns = "sales"))
val_rmse = mean_squared_error(y_true = df_test_nor["sales"],
                              y_pred = pred) ** 0.5
val_rmse 

0.08186677375964535

In [62]:
ls_k = [3, 5, 7, 9, 11]

ls_rmse = []
for k in ls_k:
    model_knn = KNeighborsRegressor(n_neighbors = k)
    model_knn.fit(X = df_train_nor.drop(columns = "sales"),
                  y = df_train_nor["sales"])
    pred = model_knn.predict(df_test_nor.drop(columns = "sales"))
    val_rmse = mean_squared_error(y_true = df_test_nor["sales"],
                                  y_pred = pred) ** 0.5
    ls_rmse = ls_rmse + [val_rmse]

In [63]:
ls_rmse

[0.08186677375964535,
 0.09879109824384892,
 0.107669855645971,
 0.11232111394853059,
 0.1136902366621185]

In [71]:
pd.Series(ls_rmse, index = ls_k)

3     0.081867
5     0.098791
7     0.107670
9     0.112321
11    0.113690
dtype: float64

In [70]:
best_k = pd.Series(ls_rmse, index = ls_k).idxmin()
best_k

3

다음은 저번달에 신규 출시된 경쟁사의 스마트폰 정보이다. 해당 스마트폰의 판매지수는 얼마로 예상되는가?  
※ 정규화 되지 않은 값으로 반올림하여 소수점 첫째 자리까지 출력하시오  
※ KNN 모델을 사용하며 이웃 개수는 직전에 최적이라고 판단한 k값을 사용하시오.  
* ROM: 256
* RAM: 6
* num_rear_camera: 4
* num_front_camera: 1
* battery_capacity: 4000
* ratings: 4.3
* sales_price: 85000
* discount_percent: 0.05
* screen_size: "Large"

In [64]:
# df_t1 = pd.DataFrame({"ROM": 256, })
df_t1 = df_test.head(1).reset_index(drop = True)
df_t1

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,5.9,256,4,2,1,2815,4.6,745,79149,0.02,0,1,0,0,0


In [65]:
df_t1["RAM"               ] = 6
df_t1["num_rear_camera"   ] = 4
df_t1["battery_capacity"  ] = 4000
df_t1["ratings"           ] = 4.3
df_t1["sales_price"       ] = 85000
df_t1["discount_percent"  ] = 0.05
df_t1["screen_size_Large" ] = 1
df_t1["screen_size_Medium"] = 0

In [66]:
df_t1

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,5.9,256,6,4,1,4000,4.3,745,85000,0.05,1,0,0,0,0


In [67]:
# MinMaxScaler().fit_transform(df_t1)
df_t1_nor = model_nor.transform(df_t1)
df_t1_nor = pd.DataFrame(df_t1_nor, columns = df_t1.columns)
df_t1_nor

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,0.011944,0.492063,0.454545,1.0,0.0,0.423077,0.625,0.001574,0.518158,0.093023,1.0,0.0,0.0,0.0,0.0


In [68]:
model_knn = KNeighborsRegressor(n_neighbors = 3)
model_knn.fit(X = df_train_nor.drop(columns = "sales"),
              y = df_train_nor["sales"])
pred = model_knn.predict(df_t1_nor.drop(columns = "sales"))
pred

array([0.00132259])

In [69]:
# model_nor.inverse_transform([pred])
df_t1_nor["sales"] = pred
pd.DataFrame(model_nor.inverse_transform(df_t1_nor), 
             columns = df_t1.columns)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,0.653333,256.0,6.0,4.0,1.0,4000.0,4.3,745.0,85000.0,0.05,1.0,0.0,0.0,0.0,0.0
